In [ ]:
import power_system_simulation as pss
import pandas as pd
from power_grid_model.utils import json_deserialize_from_file

In [ ]:
dataset = json_deserialize_from_file("input_network_data_assign3.json")

In [ ]:
reactive_power_data = pd.read_parquet("reactive_power_profile_assign3.parquet")

In [ ]:
active_power_data = pd.read_parquet("active_power_profile_assign3.parquet")

In [ ]:
# declare new PowerSimModel object
test1 = pss.PowerSim(grid_data=dataset)

# PowerSim saves the model as PowerSimModel
# access to assignment 2 functions is now given as

batch_test = test1.PowerSimModel.batch_powerflow(
    active_power_profile=active_power_data, reactive_power_profile=reactive_power_data
)
aggregate_test = test1.PowerSimModel.aggregate_voltage_table(
    active_power_profile=active_power_data, reactive_power_profile=reactive_power_data
)

# PowerSim saves the model as PowerSimModel
# access to assignment 3 functions is now
# print(test1.example_code())

In [ ]:
try:
    test1.optimal_tap_position()
except:
    print("PowerProfileNotFound: No active power profile provided.")

In [ ]:
test1.optimal_tap_position(active_power_profile=active_power_data, reactive_power_profile=reactive_power_data)

Testing if added profiles work

In [ ]:
test2 = pss.PowerSim(
    grid_data=dataset, active_power_profile=active_power_data, reactive_power_profile=reactive_power_data
)

test2.optimal_tap_position